# Parallelize a flat histogram simulation

A grand canonical ensemble transition-matrix Monte Carlo simulation is parallelized by dividing the macrostate ranges of each processor into overlapping windows.

As an exercise, compare the macrostate probability distribution output for all windows with the individual windows in the files (crit#.txt).
Attempt to unify these individual macrostate probability distributions.

In [1]:
import sys
import unittest
import multiprocessing as mp
import feasst as fst
import pyfeasst
sys.path.insert(0, fst.install_dir() + '/plugin/flat_histogram/tutorial/')
import fh

class TestFlatHistogramLJGCMCparallel(unittest.TestCase):
    def test(self):
            windows=fst.WindowExponential(fst.args({
              "alpha": "2",
              "num": "4",
              "maximum": "50",
              "extra_overlap": "2"})).boundaries()
            #windows=[[0,3],[1,16],[14,25]]
            print(windows)

            clones = fst.Clones()
            for proc, win in enumerate(windows):
                clones.add(fh.monte_carlo(proc=proc,
                    criteria=fh.criteria_flathist(macro_min=win[0], macro_max=win[1]),
                    run=False))
            clones.initialize()
            clones.run_until_complete()
            print(clones.ln_prob().values())


In [2]:
%%time
unittest.main(argv=[''], verbosity=2, exit=False)

test (__main__.TestFlatHistogramLJGCMCparallel) ... 

((0, 25), (23, 35), (33, 43), (41, 50))
(-96.9608205317803, -92.28175473702399, -88.28503947487422, -84.65898385665275, -81.33887991617596, -78.24143211664088, -75.299914328481, -72.51035737338374, -69.85882333710506, -67.29876962268315, -64.83962285110714, -62.46740610490529, -60.16466838495032, -57.94863057716537, -55.79201596752613, -53.69486847413283, -51.65002264668223, -49.66512843792507, -47.71207971878034, -45.82182792048205, -43.968214181087916, -42.15469446154233, -40.379958162713955, -38.63965075193218, -36.92196593629744, -35.23448347966558, -33.58398393400437, -31.972753878852807, -30.37018407267953, -28.790204767704335, -27.257656490255968, -25.7418978725922, -24.232834024564575, -22.756502427012556, -29.01788090859163, -19.859242402817504, -18.456435225190052, -17.074827569505192, -15.691505580594628, -14.329818318858697, -12.98294989909441, -11.65298645276876, -23.49434428081814, -9.049730284543513, -7.770769686623106, -6.49571445100011, -5.24117774190926, -3.9891736044

ok

----------------------------------------------------------------------
Ran 1 test in 13.347s

OK


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!